In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Date Alignment


In [17]:
df_trans = pd.read_csv('/content/drive/MyDrive/week 1/transformed_raw.csv')
df_apple = pd.read_csv('/content/drive/MyDrive/week 1/yfinance_data/AAPL_historical_data.csv')


df_apple['Date'] = pd.to_datetime(df_apple['Date'], format='ISO8601')
df_trans['Date'] = pd.to_datetime(df_trans['date'], format='ISO8601').dt.normalize()

df_trans['Date'] = df_trans['Date'].dt.tz_convert(None)
df_trans.drop('date', axis=1)

# print(df_trans.dtypes)
# print(df_apple.dtypes)

df_trans_apple = df_trans[df_trans['stock']=="A"]
df_apple= df_apple[(df_apple['Date'] >= df_trans['Date'].min()) & (df_apple['Date'] <= df_trans['Date'].max()) ]

print(df_trans_apple['Date'].min(), df_trans_apple['Date'].max())
print(df_apple['Date'].min(), df_apple['Date'].max())




2009-04-29 04:00:00 2020-06-05 04:00:00
2009-02-17 00:00:00 2020-06-11 00:00:00


In [20]:
df_merged = pd.merge(df_trans_apple, df_apple, on='Date', how='right')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2850 entries, 0 to 2849
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Unnamed: 0       0 non-null      float64       
 1   headline         0 non-null      object        
 2   url              0 non-null      object        
 3   publisher        0 non-null      object        
 4   date             0 non-null      object        
 5   stock            0 non-null      object        
 6   datetime         0 non-null      object        
 7   sentiment_score  0 non-null      float64       
 8   sentiment        0 non-null      object        
 9   tokens           0 non-null      object        
 10  topic            0 non-null      float64       
 11  Date             2850 non-null   datetime64[ns]
 12  Open             2850 non-null   float64       
 13  High             2850 non-null   float64       
 14  Low              2850 non-null   float64

In [5]:
df_merged.isna().sum()

,0
Unnamed: 0,2850
headline,2850
url,2850
publisher,2850
date,2850
stock,2850
datetime,2850
sentiment_score,2850
sentiment,2850
tokens,2850


In [22]:



# str_cols = df_merged.select_dtypes(include=['object']).columns
# int_cols = df_merged.select_dtypes(include=['float64']).columns

df_merged['topic']= df_merged['topic'].fillna(df_merged['topic'].mean())
df_merged['High']= df_merged['High'].fillna(df_merged['High'].mean())
df_merged['Low']= df_merged['Low'].fillna(df_merged['Low'].mean())
df_merged['Close']= df_merged['Close'].fillna(df_merged['Close'].mean())
df_merged['Open']= df_merged['Open'].fillna(df_merged['Open'].mean())
df_merged['Adj Close']= df_merged['Adj Close'].fillna(df_merged['Adj Close'].mean())
df_merged['Volume']= df_merged['Volume'].fillna(df_merged['Volume'].mean())
df_merged['Dividends']= df_merged['Dividends'].fillna(df_merged['Dividends'].mean())
df_merged['Stock Splits']= df_merged['Stock Splits'].fillna(df_merged['Stock Splits'].mean())
df_merged['sentiment_score']= df_merged['sentiment_score'].fillna(df_merged['sentiment_score'].mean())


df_merged['headline'] = df_merged['headline'].fillna(df_merged['headline'].mode())
df_merged['url'] = df_merged['url'].fillna(df_merged['url'].mode())
df_merged['publisher'] = df_merged['publisher'].fillna(df_merged['publisher'].mode())
df_merged.drop("date", axis=1)


,Unnamed: 0,headline,url,publisher,stock,datetime,sentiment_score,sentiment,tokens,topic,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-02-17,3.459643,3.465714,3.367143,3.376071,2.851008,678238400,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-02-18,3.394643,3.423214,3.311429,3.370357,2.846183,684779200,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-02-19,3.334643,3.366071,3.218214,3.237143,2.733687,922804400,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-02-20,3.192857,3.300000,3.178571,3.257143,2.750577,750316000,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-02-23,3.273214,3.285714,3.089643,3.105357,2.622398,786982000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-05,80.837502,82.937500,80.807503,82.875000,80.843407,137250400,0.0,0.0
2846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-08,82.562500,83.400002,81.830002,83.364998,81.321381,95654400,0.0,0.0
2847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-09,83.035004,86.402496,83.002502,85.997498,83.889359,147712400,0.0,0.0
2848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-10,86.974998,88.692497,86.522499,88.209999,86.047615,166651600,0.0,0.0


In [14]:
df_merged.isna().sum()

,0
Unnamed: 0,0
headline,2849
url,2155
publisher,2849
date,2849
stock,2849
datetime,2849
sentiment_score,0
sentiment,2849
tokens,2846


In [51]:
df_trans['stock'].value_counts()

,count
stock,
MRK,3333
MS,3238
NVDA,3146
MU,3142
QQQ,3106
...,...
PLW,1
RFT,1
WIP,1


In [38]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407328 entries, 0 to 1407327
Data columns (total 12 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   Unnamed: 0       1407328 non-null  int64         
 1   headline         1407328 non-null  object        
 2   url              1407328 non-null  object        
 3   publisher        1407328 non-null  object        
 4   date             1407328 non-null  object        
 5   stock            1407328 non-null  object        
 6   datetime         1407328 non-null  object        
 7   sentiment_score  1407328 non-null  float64       
 8   sentiment        1407328 non-null  object        
 9   tokens           1407328 non-null  object        
 10  topic            1407328 non-null  int64         
 11  Date             1407328 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(8)
memory usage: 128.8+ MB


In [39]:
df_apple.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2850 entries, 7109 to 9958
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          2850 non-null   datetime64[ns]
 1   Open          2850 non-null   float64       
 2   High          2850 non-null   float64       
 3   Low           2850 non-null   float64       
 4   Close         2850 non-null   float64       
 5   Adj Close     2850 non-null   float64       
 6   Volume        2850 non-null   int64         
 7   Dividends     2850 non-null   float64       
 8   Stock Splits  2850 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 222.7 KB


In [15]:
df_trans.head()

,Unnamed: 0,headline,url,publisher,date,stock,datetime,sentiment_score,sentiment,tokens,topic,Date
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 14:30:54+00:00,0.000,Neutral,"['Stocks', 'That', 'Hit', 'Week', 'Highs', 'On...",2,2020-06-05 14:30:54
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,2020-06-03 14:45:20+00:00,0.000,Neutral,"['Stocks', 'That', 'Hit', 'Week', 'Highs', 'On...",2,2020-06-03 14:45:20
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A,2020-05-26 08:30:07+00:00,0.000,Neutral,"['Biggest', 'Movers', 'From', 'Friday']",4,2020-05-26 08:30:07
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A,2020-05-22 16:45:06+00:00,0.000,Neutral,"['Stocks', 'Moving', 'In', 'Fridays', 'MidDay'...",2,2020-05-22 16:45:06
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A,2020-05-22 15:38:59+00:00,0.296,Positive,"['B', 'A', 'Securities', 'Maintains', 'Neutral...",3,2020-05-22 15:38:59


In [16]:
df_apple.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
7109,2009-02-17,3.459643,3.465714,3.367143,3.376071,2.851008,678238400,0.0,0.0
7110,2009-02-18,3.394643,3.423214,3.311429,3.370357,2.846183,684779200,0.0,0.0
7111,2009-02-19,3.334643,3.366071,3.218214,3.237143,2.733687,922804400,0.0,0.0
7112,2009-02-20,3.192857,3.300000,3.178571,3.257143,2.750577,750316000,0.0,0.0
7113,2009-02-23,3.273214,3.285714,3.089643,3.105357,2.622398,786982000,0.0,0.0
